In [ ]:
# analysis micore environment in special chromosome region
import pandas as pd
import os
import sys
import argparse

def main(argv):
    #! init
    parser = argparse.ArgumentParser(description="")
    parser.add_argument("-g", "--gene", required=True, help="The gene file")
    parser.add_argument("-c", "--chip", required=True, help="The chip file")
    parser.add_argument("-o", "--output", required=True, help='The output file')
    parser.add_argument("-l", "--length", required='True', help="The length of region to be extend")
    parser.add_argument("-n", "--number", required='True', help="The number of splits for the region")
    parser.add_argument("-cn", "--chrom_name", required='True', help="The number of chromsome")
    args = parser.parse_args()
    genef = args.gene
    chipf = args.chip
    L = int(args.length)
    N = int(args.number)
    cn = args.chrom_name
    outputf = args.output
    #! analysis
    #* 1、region extend
    df_gene = pd.read_csv(genef, sep="\t")
    df_gene = region_extend(df_gene, L)
    df_chip = pd.read_csv(chipf, sep="\t")
    df_result = pd.DataFrame()
    #* single chrom 
    #for c in ['Ghir_A01', 'Ghir_A02', 'Ghir_A03', 'Ghir_A04', 'Ghir_A05', 'Ghir_A06', 'Ghir_A07', 'Ghir_A08', 'Ghir_A09', 'Ghir_A10', 'Ghir_A11', 'Ghir_A12', 'Ghir_A13', 'Ghir_D01', 'Ghir_D02', 'Ghir_D03', 'Ghir_D04', 'Ghir_D05', 'Ghir_D06', 'Ghir_D07', 'Ghir_D08', 'Ghir_D09', 'Ghir_D10', 'Ghir_D11', 'Ghir_D12', 'Ghir_D13']:
    for c in [cn]:
        df_gene_chr = df_gene.query("chrom==@c").copy()
        df_chip_chr = df_chip.query("chrom==@c").copy()
        print(df_gene_chr)
        print(df_chip_chr)
        #* 2、region chip modified 
        df_region_chip = region_chip(df_gene_chr, df_chip_chr, L, N)
        #* 3、statistics chip modified 
        df_return = data_format(df_gene_chr, df_region_chip, N)
        df_result = pd.concat([df_result, df_return], axis=0)
    df_result.to_csv(outputf)
    
# extend the region
def region_extend(df, length):
    df['s'] = df['start'] - length
    df['e'] = df['end'] + length
    df_return = df[['chrom', 's', 'e', 'GeneId']]
    df_return.columns = ['chrom', 'start', 'end', 'GeneId']
    return df_return
# find chip modified on the region
def region_chip(df_gene, df_chip, length, number):
    df_return = pd.DataFrame()
    for index, row in df_gene.iterrows():
        ## add geneid region
        C, S, E, G = row['chrom'], row['start'], row['end'], row['GeneId']
        #print(C, S, E, G)
        df_tmp = df_chip.query("chrom==@C & start<=@E & end>=@S").copy()
        df_tmp.reset_index(inplace=True, drop=True)
        df_tmp.loc[0, 'start'] , df_tmp.loc[df_tmp.shape[0]-1, 'end']= S, E
        df_tmp['GeneId'] = [G]*df_tmp.shape[0]
        #print(df_tmp)
        #df_tmp['Total_Length'] = [E-S + 2*length]*df_tmp.shape[0]
        ## split region and add tags
        average_length = int((E-S)/number)
        gene_region_length = E-S
        #print(average_length)
        for i in range(number):
            S_tmp = S
            E_tmp = S + average_length
            df_tmp2 = df_tmp.query("chrom==@C & start <= @E_tmp & end>= @S_tmp").copy()
            df_tmp2.reset_index(drop=True, inplace=True)
            df_tmp2.loc[0, 'start'] = S_tmp
            df_tmp2.loc[df_tmp2.shape[0]-1, 'end'] = E_tmp
            tags = "R{}".format(i)
            df_tmp2['tags'] = [tags]*df_tmp2.shape[0]
            df_tmp2['gene_region_length'] = [gene_region_length]*df_tmp2.shape[0]
            df_tmp2['sig_total_length'] = [E_tmp-S_tmp]*df_tmp2.shape[0]
            df_tmp2['length'] = df_tmp2['end'] - df_tmp2['start'] 
            df_return = pd.concat([df_return , df_tmp2], axis=0)
            S = E_tmp+1
    return df_return
# adjust data format
def data_format(df_gene, df_chip, number):
    df_tmp = df_chip.groupby(by=['GeneId','gene_region_length', 'sig_total_length', 'tags', 'sigs'], as_index=False)['length'].sum()
    df_tmp['ratio'] = df_tmp['length']/df_tmp['sig_total_length']
    #print(df_tmp.head(30))
    df_tags = df_gene[['GeneId']].copy()
    # tags
    for i in range(number):
        t = 'R{}'.format(i)
        df_tags[t] = [t]*df_tags.shape[0]
    df_tags = pd.melt(df_tags, id_vars=['GeneId'])
    df_tags = df_tags[['GeneId', 'variable']].copy()
    df_tags.columns = ['GeneId', 'tags']
    # sigs
    df_sigs = df_gene[['GeneId']].copy()
    for i in range(1,5):
        s = "E{}".format(i)
        df_sigs[s] = [s]*df_sigs.shape[0]
    df_sigs = pd.melt(df_sigs, id_vars=['GeneId'])
    df_sigs = df_sigs[['GeneId', 'variable']].copy()
    df_sigs.columns = ['GeneId', 'sigs']
    df_gene_tags = pd.merge(df_tags, df_sigs, how='outer', on=['GeneId'])
    df_gene_tags['ratio'] = [0]*df_gene_tags.shape[0]
    df_tmp = df_tmp[['GeneId', 'tags', 'sigs', 'ratio']]
    df_tmp = pd.concat([df_tmp, df_gene_tags], axis=0)
    df_tmp.drop_duplicates(keep='first', inplace=True, subset=['GeneId', 'tags', 'sigs'])
    df_tmp.sort_values(by=['GeneId', 'tags', 'sigs'], inplace=True)
    df_tmp['tags_sigs'] = df_tmp['tags'].map(str) + "-" + df_tmp['sigs'].map(str)
    df_tmp = df_tmp[['GeneId', 'tags_sigs', 'ratio']]
    df_return = df_tmp.pivot(index='GeneId', columns='tags_sigs', values='ratio')
    return df_return
if __name__ == "__main__":
    main(sys.argv[1:])

In [58]:
## 将多个数据文件整理为一个文件，并且z-score 标准化，再用umap 来聚类分析

# 1、数据合并
samples = ['hypocotyl', 'leaf', 'radicle', 'stigma']
#samples = ['hypocotyl']
chroms = ['Ghir_A01', 'Ghir_A02', 'Ghir_A03', 'Ghir_A04', 'Ghir_A05', 'Ghir_A06', 'Ghir_A07', 'Ghir_A08', 'Ghir_A09', 'Ghir_A10', 'Ghir_A11', 'Ghir_A12', 'Ghir_A13', 
          'Ghir_D01', 'Ghir_D02', 'Ghir_D03', 'Ghir_D04', 'Ghir_D05', 'Ghir_D06', 'Ghir_D07', 'Ghir_D08', 'Ghir_D09', 'Ghir_D10', 'Ghir_D11', 'Ghir_D12', 'Ghir_D13']
df_micro_state = pd.DataFrame()
for s in samples:
    df_tmp = pd.DataFrame()
    for c in chroms:
        input_f = "G:/Billfish/J668_multip_tissue_3D-genome/Chip_Seq/micro_state/chromHMM_ratio/5/{0}_micro_state_{1}.csv".format(s, c)
        df = pd.read_csv(input_f)
        df_tmp = pd.concat([df_tmp, df], axis=0)
        df_tmp.reset_index(drop=True, inplace=True)
    if df_micro_state.shape[0] == 0:
        df_micro_state = df_tmp.copy()
    else:
        df_micro_state = pd.merge(df_micro_state, df_tmp, how='inner', on=['GeneId'])
        #df_micro_state = pd.merge(df_micro_state, df_tmp, how='inner', on=['GeneId'])
#print(df_micro_state)

# z-score 标准化， umap 聚类
#print(df_micro_state)
col_list = []
for s in samples:
    for k in range(1,5):
        for i in range(5):
            col = "{0}_R{1}-E{2}".format(s, k, i)
            col_list.append(col)

def zscore_umap(df):
    import scipy.stats as stats
    from sklearn.preprocessing import StandardScaler
    import umap.umap_ as umap
    #z-score
    id_list = df['GeneId'].tolist()
    df.set_index('GeneId', inplace=True)
    df_numpy = df.to_numpy()
    # df_zscore = stats.zscore(df_numpy, axis=1)
    # df_zscore = pd.DataFrame(df_zscore, columns=col_list)
    df_zscore = pd.DataFrame(df_numpy, columns=col_list)
    # umap
    reducer = umap.UMAP()
    scaled_penguin_data = StandardScaler().fit_transform(df_zscore)
    embedding = reducer.fit_transform(scaled_penguin_data)
    df_umap = pd.DataFrame(embedding, columns =['umap1', 'umap2'], index=id_list)
    return df_umap
df_umap = zscore_umap(df_micro_state)

C:\Users\pc\AppData\Local\Temp\ipykernel_22872\2494101199.py:19: FutureWarning: Passing 'suffixes' which cause duplicate columns {'R1-E1_x', 'R4-E4_x', 'R2-E1_x', 'R1-E3_x', 'R2-E2_x', 'R2-E4_x', 'R4-E2_x', 'R4-E1_x', 'R1-E4_x', 'R3-E2_x', 'R1-E2_x', 'R0-E4_x', 'R0-E3_x', 'R3-E4_x', 'R0-E2_x', 'R0-E1_x', 'R3-E3_x', 'R3-E1_x', 'R2-E3_x', 'R4-E3_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_micro_state = pd.merge(df_micro_state, df_tmp, how='inner', on=['GeneId'])


In [57]:
df_umap.reset_index(inplace=True)
df_umap.columns = ['GeneId', 'umap1', 'umap2']
df_gene = pd.read_csv('G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/expression_gene/J668_all_tissue_average_TPM.txt', sep="\t")
df_gene = df_gene[['GeneId', 'Hypocotyl']]
df_result = pd.merge(df_umap, df_gene, how='inner', on=['GeneId'])
# df_gene = df_result['Hypocotyl'].to_numpy()
# df_gene_zscore = stats.zscore(df_gene, axis=0)
# df_result['Hypocotyl'] = df_gene_zscore
df_result.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/Chip_Seq/micro_state/gene/hypocotyl_gene_micro_state.csv", index=False)

In [59]:
### 将TSG 选出来，umap 聚类的结果
df_umap.reset_index(inplace=True)
df_umap.columns = ['GeneId', 'umap1', 'umap2']
print(df_umap)
df_TSG = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/tau_gini/J668_tissue_special_gene.txt", sep="\t")
df_TSG_id = df_TSG.query("Tags=='hypocotyl' | Tags=='leaf' | Tags=='radicle' | Tags=='stigma'")[['GeneId', 'Tags']].copy()
df_TSG_umap = pd.merge(df_TSG_id, df_umap, how='inner', on=['GeneId'])
df_TSG_umap.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/Chip_Seq/micro_state/TSG/result.csv", index=False)


                GeneId      umap1     umap2
0      Ghir_A01G000010   6.050754  7.385537
1      Ghir_A01G000020   6.954064  1.978401
2      Ghir_A01G000030   6.013783  5.801857
3      Ghir_A01G000040   9.074519  9.409721
4      Ghir_A01G000050  13.280522  6.789733
...                ...        ...       ...
68895  Ghir_D13G025640  13.688496  6.244218
68896  Ghir_D13G025650  11.114699  6.351100
68897  Ghir_D13G025660   9.548344  9.074274
68898  Ghir_D13G025670  10.023056  1.930667
68899  Ghir_D13G025680   6.016584  7.462289

[68900 rows x 3 columns]


In [ ]:
# 分析bam文件中的信号在基因处的分布
# 分析基因所处于的bam区间
#Nutures
#20221101
#Analysis the gene located in the bins

import pandas as pd
import sys

gene_f = sys.argv[1]
bin_f = sys.argv[2]
output_f = sys.argv[3]
def gene_bin(gene, bins, output):
    df_gene = pd.read_csv(gene, sep="\t", names=['chrom', 'start', 'end', 'GeneId'])
    df_bin = pd.read_csv(bins, sep="\t", names=['chrom', 'start', 'end', 'signal'])
    chrom = ['Ghir_A01', 'Ghir_A02', 'Ghir_A03', 'Ghir_A04', 'Ghir_A05', 'Ghir_A06', 'Ghir_A07', 'Ghir_A08', 'Ghir_A09', 'Ghir_A10', 'Ghir_A11', 'Ghir_A12', 'Ghir_A13',
             'Ghir_D01', 'Ghir_D02', 'Ghir_D03', 'Ghir_D04', 'Ghir_D05', 'Ghir_D06', 'Ghir_D07', 'Ghir_D08', 'Ghir_D09', 'Ghir_D10', 'Ghir_D11', 'Ghir_D12', 'Ghir_D13']
    df_return = pd.DataFrame()
    for c in chrom:
        df_gene_chr = df_gene.query("chrom==@c").copy()
        df_bin_chr = df_bin.query("chrom==@c").copy()
        for index, row in df_gene_chr.iterrows():
            S, E , G = row['start'], row['end'], row['GeneId']
            df_tmp = df_bin_chr.query("start<=@S & end>=@E").copy()
            df_tmp['GeneId'] = [G]*df_tmp.shape[0]
            df_return = pd.concat([df_return, df_tmp], axis=0)
    df_return['Start'] = df_return['start'] - 3000
    df_return['End'] = df_return['end'] + 3000
    df_return.to_csv(output, sep="\t", columns=['chrom', 'Start', 'End'], index=False)
gene_bin(gene_f, bin_f, output_f)

In [ ]:
## 分析基因周围的chip信号丰度
import pandas as pd
import sys
gene_f = sys.argv[1]
single_f = sys.argv[2]
output_f = sys.argv[3]

def epigenome_single(gene, single, output):
    df_gene = pd.read_csv(gene_f, sep="\t", names=['chrom', 'start', 'end', 'sigs'])
    df_single = pd.read_csv(single_f, sep="\t", names=['chrom', 'start', 'end', 'single'])
    df_gene[['GeneId', 'Tags']] = df_gene['sigs'].split("-", expand = True)
    df_gene = df_gene[['chrom', 'start', 'end', 'GeneId', 'Tags']].copy()
    df_tmp = pd.merge(df_gene, df_single, how='inner', on=['chrom', 'start', 'end', 'GeneId'])
    df_return = df_tmp.pivot(columns='Tags', index='GeneId', values='single')
    df_return.to_csv(output, index=False)
epigenome_single(gene_f, single_f, output_f)

In [59]:
## 对结果zscore 和umap 标准化
import pandas as pd
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import umap.umap_ as umap
def zscore_umap(df):
    #z-score
    id_list = df['GeneId'].tolist()
    df.set_index('GeneId', inplace=True)
    df[df<=0] = 0
    df.fillna(0, inplace=True)
    df_numpy = df.to_numpy()
    #print(df_numpy)
    # df_zscore = stats.zscore(df_numpy, axis=1)
    # df_zscore = pd.DataFrame(df_zscore)
    # df_zscore.fillna(0, inplace=True)
    df_zscore = pd.DataFrame(df_numpy)
    # # umap
    reducer = umap.UMAP()
    scaled_penguin_data = StandardScaler().fit_transform(df_zscore)
    embedding = reducer.fit_transform(scaled_penguin_data)
    df_umap = pd.DataFrame(embedding, columns =['umap1', 'umap2'], index=id_list)
    return df_umap
# df = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/Chip_Seq/micro_state/epigenome_single/gene_cotyledon_H3K27ac_3K_100bp.csv")
# #print(df.head())
# df_gene = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/expression_gene/J668_all_tissue_average_TPM.txt", sep="\t")
# df_umap = zscore_umap(df)
# df_umap.reset_index(inplace=True)
# df_umap.columns = ['GeneId', 'umap1', 'umap2']
# df_result = pd.merge(df_umap, df_gene, how='inner', on=['GeneId'])
# df_result.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/Chip_Seq/micro_state/epigenome_single/Gene_cotyledon_H3K27ac.csv", index=False)

In [53]:
## 整理，将H3K4me3 和 H3K27ac 都加进去
import pandas as pd
df_H3K4me3 = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/Chip_Seq/micro_state/epigenome_single/gene_cotyledon_H3K4me3_3K_100bp.csv")
df_H3K27ac = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/Chip_Seq/micro_state/epigenome_single/gene_cotyledon_H3K27ac_3K_100bp.csv")
df = pd.merge(df_H3K4me3, df_H3K27ac, how='inner', on=['GeneId'])
df_gene = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/expression_gene/J668_all_tissue_average_TPM.txt", sep="\t")
df_umap = zscore_umap(df)
df_umap.reset_index(inplace=True)
df_umap.columns = ['GeneId', 'umap1', 'umap2']
df_result = pd.merge(df_umap, df_gene, how='inner', on=['GeneId'])
df_result.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/Chip_Seq/micro_state/epigenome_single/Gene_cotyledon_H3K4me3_H3K27ac.csv", index=False)

In [68]:
## 整理，将H3K4me3 
import pandas as pd
df= pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/Chip_Seq/micro_state/epigenome_single/gene_cotyledon_H3K4me3_3K_100bp.csv")
df_gene = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/expression_gene/J668_all_tissue_average_TPM.txt", sep="\t")
# df_gene.reset_index(inplace=True)
# df_gene.rename(columns={'index':'GeneId'}, inplace=True)
df_umap = zscore_umap(df)
df_umap.reset_index(inplace=True)
df_umap.columns = ['GeneId', 'umap1', 'umap2']
df_result = pd.merge(df_umap, df_gene, how='inner', on=['GeneId'])
df_result.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/Chip_Seq/micro_state/epigenome_single/Gene_cotyledon_H3K4me3.csv", index=False)

In [55]:
## 整理，将H3K27ac
import pandas as pd
df= pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/Chip_Seq/micro_state/epigenome_single/gene_cotyledon_H3K27ac_3K_100bp.csv")
df_gene = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/expression_gene/J668_all_tissue_average_TPM.txt", sep="\t")
df_umap = zscore_umap(df)
df_umap.reset_index(inplace=True)
df_umap.columns = ['GeneId', 'umap1', 'umap2']
df_result = pd.merge(df_umap, df_gene, how='inner', on=['GeneId'])
df_result.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/Chip_Seq/micro_state/epigenome_single/Gene_cotyledon_H3K27ac.csv", index=False)

In [34]:
## 对于非编码区的功能调控元件
## 将多个数据文件整理为一个文件，并且z-score 标准化，再用umap 来聚类分析

# 1、数据合并
#samples = ['hypocotyl', 'leaf', 'radicle', 'stigma', 'cotyledon']
samples = ['hypocotyl']

#samples = ['hypocotyl']
chroms = ['Ghir_A01', 'Ghir_A02', 'Ghir_A03', 'Ghir_A04', 'Ghir_A05', 'Ghir_A06', 'Ghir_A07', 'Ghir_A08', 'Ghir_A09', 'Ghir_A10', 'Ghir_A11', 'Ghir_A12', 'Ghir_A13', 
          'Ghir_D01', 'Ghir_D02', 'Ghir_D03', 'Ghir_D04', 'Ghir_D05', 'Ghir_D06', 'Ghir_D07', 'Ghir_D08', 'Ghir_D09', 'Ghir_D10', 'Ghir_D11', 'Ghir_D12', 'Ghir_D13']
df_micro_state = pd.DataFrame()
for s in samples:
    df_tmp = pd.DataFrame()
    for c in chroms:
        input_f = "G:/Billfish/J668_multip_tissue_3D-genome/ATAC/PETRE/5/{0}_micro_state_{1}.csv".format(s, c)
        df = pd.read_csv(input_f)
        df_tmp = pd.concat([df_tmp, df], axis=0)
        df_tmp.reset_index(drop=True, inplace=True)
    if df_micro_state.shape[0] == 0:
        df_micro_state = df_tmp.copy()
    else:
        df_micro_state = pd.merge(df_micro_state, df_tmp, how='inner', on=['GeneId'])
        #df_micro_state = pd.merge(df_micro_state, df_tmp, how='inner', on=['GeneId'])
#print(df_micro_state)
df_micro_state.fillna(0, inplace=True)
# z-score 标准化， umap 聚类
#print(df_micro_state)
col_list = []
for s in samples:
    for k in range(1,5):
        for i in range(5):
            col = "{0}_R{1}-E{2}".format(s, k, i)
            col_list.append(col)

def zscore_umap(df):
    import scipy.stats as stats
    from sklearn.preprocessing import StandardScaler
    import umap.umap_ as umap
    #z-score
    id_list = df['GeneId'].tolist()
    df.set_index('GeneId', inplace=True)
    df_numpy = df.to_numpy()
    # df_zscore = stats.zscore(df_numpy, axis=1)
    # df_zscore = pd.DataFrame(df_zscore, columns=col_list)
    df_zscore = pd.DataFrame(df_numpy, columns=col_list)
    # umap
    reducer = umap.UMAP()
    scaled_penguin_data = StandardScaler().fit_transform(df_zscore)
    embedding = reducer.fit_transform(scaled_penguin_data)
    df_umap = pd.DataFrame(embedding, columns =['umap1', 'umap2'], index=id_list)
    return df_umap
df_umap = zscore_umap(df_micro_state)
df_umap.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/ATAC/PETRE/PETRE_micro_state.csv")


In [18]:
# 将umap结果与平均信号结合起来
import pandas as pd
import scipy.stats as stats
df_single = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/Chip_Seq/micro_state/epigenome_single/Gene_cotyledon_H3K4me3_Gene.csv")
df_umap = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/Chip_Seq/micro_state/epigenome_single/gene_cotyledon_H3K27ac_3K_100bp.csv", index_col=0)
df_umap['average'] = df_umap.median(axis=1)
df_umap.reset_index(inplace=True)
df_single = df_single[['GeneId', 'umap1', 'umap2']]
df_umap = df_umap[['GeneId', 'average']].copy()
df_result = pd.merge(df_single, df_umap, how='inner', on=['GeneId'])
## 将小于0的和空值替换为0
df_result_tmp = df_result.query("average<0").copy()
df_result_tmp['average'] = 0
df_result = pd.concat([df_result, df_result_tmp], axis=0)
df_result.fillna(0, inplace=True)
df_result.drop_duplicates(keep='last', inplace=True, subset=['GeneId'])
# 将数据z-score 标准化
df_result['average_zscore'] = stats.zscore(df_result['average'], axis=0)
# 将数据归一化到0-1之间
max_value = df_result['average_zscore'].max()
min_value = df_result['average_zscore'].min()
df_result['normal'] = (df_result['average_zscore']-min_value)/(max_value-min_value)
df_result['normal'].replace(0, 0.00001, inplace=True)
df_result.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/Chip_Seq/micro_state/epigenome_single/gene_cotyledon_H3K4me3_3K_100bp_chip.csv", index=False)

In [33]:
## 将hypocotyl 和stigma 结合起来， 分析TSG 是否具有H3K27ac 的偏好性
import pandas as pd
df_cotyledon = pd.read_csv('G:/Billfish/J668_multip_tissue_3D-genome/Chip_Seq/micro_state/epigenome_single/cotyledon/gene_cotyledon_H3K4me3_3K_100bp.csv')
df_stigma = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/Chip_Seq/micro_state/epigenome_single/stigma/gene_stigma_H3K4me3_3K_100bp.csv")
df = pd.merge(df_cotyledon, df_stigma, how='inner', on=['GeneId'])
df.fillna(0, inplace=True)
def zscore_umap(df):
    import scipy.stats as stats
    from sklearn.preprocessing import StandardScaler
    import umap.umap_ as umap
    #z-score
    df.set_index('GeneId', inplace=True)
    df = df.loc[~(df==0).all(axis=1), :]
    id_list = df.index.to_list()
    df_numpy = df.to_numpy()
    df_zscore = stats.zscore(df_numpy, axis=1)
    df_zscore = pd.DataFrame(df_zscore)
    #df_zscore = pd.DataFrame(df_numpy)
    # umap
    reducer = umap.UMAP()
    scaled_penguin_data = StandardScaler().fit_transform(df_zscore)
    embedding = reducer.fit_transform(scaled_penguin_data)
    df_umap = pd.DataFrame(embedding, columns =['umap1', 'umap2'], index=id_list)
    return df_umap
df_normal = zscore_umap(df)
df_normal.reset_index(inplace=True)
df_normal.rename(columns={'index': 'GeneId'}, inplace=True)
print(df_normal)
# ## 打上TSG 标签
df_tsg = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/tau_gini/J668_tissue_special_geneid.txt", sep="\t")
df_tsg_cotyledon_stigam = df_tsg.query("tags=='cotyledon' | tags=='stigma'")[['GeneId', 'tags']].copy()
df_result = pd.merge(df_normal, df_tsg_cotyledon_stigam, how='inner', on=['GeneId'])
df_result.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/Chip_Seq/micro_state/TSG/cotyledon_hypocotyl_tsg.csv", index=False)

                GeneId      umap1     umap2
0      Ghir_A01G000010   9.311453  5.451591
1      Ghir_A01G000020  10.173335  8.722606
2      Ghir_A01G000030   3.787272  7.875386
3      Ghir_A01G000040  11.109644  3.293409
4      Ghir_A01G000050   5.330428  2.857170
...                ...        ...       ...
68894  Ghir_D13G025640   0.587722  3.416919
68895  Ghir_D13G025650   4.998712  2.426601
68896  Ghir_D13G025660  10.901816  3.161812
68897  Ghir_D13G025670  -1.615474  4.846691
68898  Ghir_D13G025680  -0.376509  8.412956

[68899 rows x 3 columns]
